# Accessing model data from CEDA archives 

This notebook accesses precipitation and temperature data from first ensemble member for a number of CMIP6 models from the CEDA archives and stores them locally as nc files to be accessed by other notebooks for analysis.

In [2]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cftime
from xmip.preprocessing import rename_cmip6
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

/home/users/nanaokan/.conda/envs/cmipv2/lib/python3.13/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)


In [3]:


# PRECIP


#list centres - precip
cents_list_s2 = os.listdir('/badc/cmip6/data/CMIP6/ScenarioMIP/')
cents_list_s2.remove('DWD') #doesnt have ssp245
cents_list_s2.remove('HAMMOZ-Consortium') #doesnt have ssp245
cents_list_s2.remove('IPSL') # doesn't have Amon
cents_list_s2.remove('UA') # dimensions are weird 
cents_list_s2.remove('CNRM-CERFACS') # s245 only runs until 2020
#cents_list_s2.remove('NASA-GISS') # only has tas no precip
cents_list_s2.remove('MOHC')#cause hardcoding these two for correct model
cents_list_s2.remove('NCAR') #cause hardcoding these two for correct model
cents_list_s2.remove('THU') # values are 1000x smaller than others
cents_list_s2.remove('EC-Earth-Consortium') # starts 1975: no PI

#list centres 
cents_list_hist = os.listdir('/badc/cmip6/data/CMIP6/CMIP/')
cents_list_hist.remove('UA') # dimensions are weird
cents_list_hist.remove('IPSL') # doesn't have Amon
cents_list_hist.remove('CNRM-CERFACS') # s245 only runs until 2020
cents_list_hist.remove('MOHC')#cause hardcoding these two for correct model
cents_list_hist.remove('NCAR') #cause hardcoding these two for correct model


#list models(?)
mods_list_s2 = []
for c in cents_list_s2:
    mods = os.listdir(f'/badc/cmip6/data/CMIP6/ScenarioMIP/{c}/')
    if len(mods) >= 1:
        mods_list_s2.append(mods[0])
    elif len(mods) < 1:
        print('THIS DOES NOT HAVE MODELS')

#add the missing ones - surpassing loop:
#cents_list_s2.append('UA')
#mods_list_s2.append('IPSL-CM6A-LR')

#list models(?)
mods_list_hist = []
for c in cents_list_hist:
    mods = os.listdir(f'/badc/cmip6/data/CMIP6/CMIP/{c}/')
    if len(mods) >= 1:
        mods_list_hist.append(mods[0])
    elif len(mods) < 1:
        print('THIS DOES NOT HAVE MODELS')


mods_remove = []
cents_remove = []

for x in range(len(mods_list_s2)):
    if not mods_list_s2[x] in mods_list_hist:
        mods_remove.append(mods_list_s2[x])
        cents_remove.append(cents_list_s2[x])
for x in range(len(cents_remove)):
    cents_list_s2.remove(cents_remove[x])
    mods_list_s2.remove(mods_remove[x])
    
mods_remove = []
cents_remove = []
for x in range(len(mods_list_hist)):
    if not mods_list_hist[x] in mods_list_s2:
        mods_remove.append(mods_list_hist[x])
        cents_remove.append(cents_list_hist[x])
for x in range(len(cents_remove)):
    cents_list_hist.remove(cents_remove[x])
    mods_list_hist.remove(mods_remove[x])

mods_list = []
cents_list = []
for x in range(len(mods_list_s2)):
    if mods_list_s2[x] in mods_list_hist:
        mods_list.append(mods_list_s2[x])
        cents_list.append(cents_list_s2[x])

#list ensemble members
'''

ens_list_s2 = []

if len(cents_list) == len(mods_list):
    for i in range(len(cents_list)):
        ens = os.listdir(f'/badc/cmip6/data/CMIP6/ScenarioMIP/{cents_list[i]}/{mods_list[i]}/ssp245/')
        if len(ens) >= 1:
            for e in range(len(ens)):
                if not '.' in ens[e]:
                    ens_list_s2.append(ens[e])
                    break
        elif len(ens) < 1:
            print('THIS DOES NOT HAVE ENSEMBLE MEMBERS')
else: print('CENTRES AND MODELS NOT SAME LENGTH')

'''

ens_list_s2 = []

if len(cents_list) == len(mods_list):
    for i in range(len(cents_list)):
        ens = os.listdir(f'/badc/cmip6/data/CMIP6/ScenarioMIP/{cents_list[i]}/{mods_list[i]}/ssp245/')
        if len(ens) >= 1:
            for e in range(len(ens)):
                if 'r1i' in ens[e]:
                    ens_list_s2.append(ens[e])
                    break
        elif len(ens) < 1:
            print('THIS DOES NOT HAVE ENSEMBLE MEMBERS')
else: print('CENTRES AND MODELS NOT SAME LENGTH')



#list ensemble members
#txt_list = []
ens_list_hist = []
if len(cents_list) == len(mods_list):
    for i in range(len(cents_list)):
        ens = os.listdir(f'/badc/cmip6/data/CMIP6/CMIP/{cents_list[i]}/{mods_list[i]}/historical/')
        if ens_list_s2[i] in ens:
            ens_list_hist.append(ens_list_s2[i])
        else:
            print(i)
ens_list = ens_list_hist

mods_list_pr, cents_list_pr, ens_list_pr = mods_list, cents_list, ens_list


#actually retrieve them

ds_list_s2 = []
ds_list_hist = []

dss = [ds_list_s2, ds_list_hist]
scen = ['ssp245', 'historical']

ds_long = []

for x in range(2):
    
    for i in range(len(cents_list)):
       # print(i)
        path = f'/badc/cmip6/data/CMIP6/*MIP/{cents_list[i]}/{mods_list[i]}/{scen[x]}/{ens_list[i]}/Amon/pr/*/latest/'
        ds = rename_cmip6(xr.open_mfdataset(path+'*.nc'))
        dss[x].append(ds)
    #    print(dss[x][i].institution_id)

    #adding UKESM and CESM at the end 
    path = f'/badc/cmip6/data/CMIP6/*MIP/MOHC/UKESM1-0-LL/{scen[x]}/r10i1p1f2/Amon/pr/gn/latest/'
    ds = rename_cmip6(xr.open_mfdataset(path+'*.nc'))
    dss[x].append(ds)
    
    path = f'/badc/cmip6/data/CMIP6/*MIP/NCAR/CESM2-WACCM/{scen[x]}/r1i1p1f1/Amon/pr/gn/latest/'
    ds = rename_cmip6(xr.open_mfdataset(path+'*.nc'))
    dss[x].append(ds)


for i in range(len(dss[0])):
    DS_pr =  xr.concat([dss[1][i], dss[0][i]], dim='time')
    ds_long.append(DS_pr)


def get_TPC(da_pr):
    """ 
    - Global returns 10yr avg of global precip for each lon-lat point
    - Tropical returns 10yr avg of tropical precip for each lon-lat point 
    - TPC returns the latitude defining two regions of equal area-integrated 
    precipitation between two tropical latitude boundaries, 10 yr avg """
    
    # global
    pr_yr_glob = da_pr.groupby('time.year').mean('time')
    pr_10_glob = pr_yr_glob.rolling(year=10, center=True).mean().dropna("year") * 86400 # cause s to day 
 #   pr_10_glob_mean = pr_10_glob.mean(dim = ['x', 'y'])

    #tropical
    da_pr_trop = da_pr.sel(y=slice(-20, 20))
    pr_yr_trop = da_pr_trop.groupby('time.year').mean('time')
    pr_10_trop = pr_yr_trop.rolling(year=10, center=True).mean().dropna("year") * 86400 # cause s to day 
  #  pr_10_trop_mean = pr_10_trop.mean(dim = ['x','y'])

    #centroid
    weights = np.cos(np.deg2rad(da_pr.y))
    da_pr_zonal = da_pr_trop.mean(dim='x')
    da_pr_zonal_weighted = da_pr_zonal*weights
    pr_weighted_lats = da_pr_trop.y.weighted(da_pr_zonal_weighted)
    cr = pr_weighted_lats.mean(dim='y')
    cr_yr = cr.groupby('time.year').mean('time')
    cr_10 = cr_yr.rolling(year=10, center=True).mean().dropna("year")
    
    return pr_10_glob, pr_10_trop, cr_10


glob_pr = []
trop_pr = []
centroids = []
for i in range(len(ds_long)):
    glob, trop, cr = get_TPC(ds_long[i].pr)
    glob_pr.append(glob)
    trop_pr.append(trop)
    centroids.append(cr)

data_list = [glob_pr, trop_pr, centroids]
names = ['glob_pr', 'trop_pr', 'centroids']

# Define output directory
output_dir = "inter_files_precip"
os.makedirs(output_dir, exist_ok=True)  # Create the folder if it doesn't exist

# Save each DataArray in the list
for data in range(len(data_list)):
    dirpath = os.path.join(output_dir, names[data])
    os.makedirs(dirpath, exist_ok=True)
    for i in range(len(data_list[data])):
        filename = f"{names[data]}_{i+1}.nc"
        filepath = os.path.join(dirpath, filename)
    
        # Save the DataArray to NetCDF format
        data_list[data][i].to_netcdf(filepath)
'''
data_list = [patterns]
names = ['patterns']

output_dir = "inter_files"
os.makedirs(output_dir, exist_ok=True)  # Create the folder if it doesn't exist

# Save each DataArray in the list
for data in range(len(data_list)):
    dirpath = os.path.join(output_dir, names[data])
    os.makedirs(dirpath, exist_ok=True)
    for i in range(len(data_list[data])):
        filename = f"{names[data]}_{i+1}.nc"
        filepath = os.path.join(dirpath, filename)
    
        # Save the DataArray to NetCDF format
        data_list[data][i].to_netcdf(filepath)
'''

/tmp/ipykernel_1209/2496359610.py:142: SerializationWarning: Unable to decode time axis into full numpy.datetime64[ns] objects, continuing using cftime.datetime objects instead, reason: dates out of range. To silence this warning use a coarser resolution 'time_unit' or specify 'use_cftime=True'.
  ds = rename_cmip6(xr.open_mfdataset(path+'*.nc'))
/tmp/ipykernel_1209/2496359610.py:142: SerializationWarning: Unable to decode time axis into full numpy.datetime64[ns] objects, continuing using cftime.datetime objects instead, reason: dates out of range. To silence this warning use a coarser resolution 'time_unit' or specify 'use_cftime=True'.
  ds = rename_cmip6(xr.open_mfdataset(path+'*.nc'))
/tmp/ipykernel_1209/2496359610.py:142: SerializationWarning: Unable to decode time axis into full numpy.datetime64[ns] objects, continuing using cftime.datetime objects instead, reason: dates out of range. To silence this warning use a coarser resolution 'time_unit' or specify 'use_cftime=True'.
  ds 

'\ndata_list = [patterns]\nnames = [\'patterns\']\n\noutput_dir = "inter_files"\nos.makedirs(output_dir, exist_ok=True)  # Create the folder if it doesn\'t exist\n\n# Save each DataArray in the list\nfor data in range(len(data_list)):\n    dirpath = os.path.join(output_dir, names[data])\n    os.makedirs(dirpath, exist_ok=True)\n    for i in range(len(data_list[data])):\n        filename = f"{names[data]}_{i+1}.nc"\n        filepath = os.path.join(dirpath, filename)\n\n        # Save the DataArray to NetCDF format\n        data_list[data][i].to_netcdf(filepath)\n'

In [13]:
patterns = []

for ds in range(len(ds_list_hist)):
    ds_con = xr.concat([ds_list_hist[ds], ds_list_s2[ds]], dim = 'time')
    ds_con_yrly = ds_con.groupby('time.year').mean('time')
    pattern = ds_con_yrly.rolling(year=10, center=True).mean().dropna("year")
    patterns.append(pattern)
    
'''
patterns_mean = []
for i in patterns:
    means = i.rolling(year=10, center=True).mean().dropna("year")
    patterns_mean.append(means)
'''

data_list = [patterns]
names = ['patterns_precip']

output_dir = "inter_files_precip"
os.makedirs(output_dir, exist_ok=True)  # Create the folder if it doesn't exist

# Save each DataArray in the list
for data in range(len(data_list)):
    dirpath = os.path.join(output_dir, names[data])
    os.makedirs(dirpath, exist_ok=True)
    for i in range(len(data_list[data])):
        filename = f"{names[data]}_{i+1}.nc"
        filepath = os.path.join(dirpath, filename)
    
        # Save the DataArray to NetCDF format
        data_list[data][i].to_netcdf(filepath)


In [ ]:

#### TAS  


#list centres for tas 
cents_list_s2 = os.listdir('/badc/cmip6/data/CMIP6/ScenarioMIP/')
cents_list_s2.remove('DWD') #doesnt have ssp245, loser 
cents_list_s2.remove('HAMMOZ-Consortium') #doesnt have ssp245, loser 
cents_list_s2.remove('IPSL') # doesn't have Amon
cents_list_s2.remove('UA') # dimensions are weird 
cents_list_s2.remove('CNRM-CERFACS') # s245 only runs until 2020
cents_list_s2.remove('MOHC')#cause hardcoding these two for correct model
cents_list_s2.remove('NCAR') #cause hardcoding these two for correct model
#cents_list_s2.remove('NASA-GISS') # only has tas no precip
cents_list_s2.remove('THU') # precip values are 1000x smaller than others
cents_list_s2.remove('EC-Earth-Consortium') # starts 1975: no PI base 

#list centres - HIST
cents_list_hist = os.listdir('/badc/cmip6/data/CMIP6/CMIP/')
cents_list_hist.remove('UA') # dimensions are weird
cents_list_hist.remove('IPSL') # doesn't have Amon
cents_list_hist.remove('CNRM-CERFACS') # s245 only runs until 2020
cents_list_hist.remove('MOHC')#cause hardcoding these two for correct model
cents_list_hist.remove('NCAR') #cause hardcoding these two for correct model


#list models(?)
mods_list_s2 = []
for c in cents_list_s2:
    mods = os.listdir(f'/badc/cmip6/data/CMIP6/ScenarioMIP/{c}/')
    if len(mods) >= 1:
        mods_list_s2.append(mods[0])
    elif len(mods) < 1:
        print('THIS DOES NOT HAVE MODELS')

#add the missing ones - surpassing loop:
cents_list_s2.append('UA')
mods_list_s2.append('IPSL-CM6A-LR')

#list models(?)
mods_list_hist = []
for c in cents_list_hist:
    mods = os.listdir(f'/badc/cmip6/data/CMIP6/CMIP/{c}/')
    if len(mods) >= 1:
        mods_list_hist.append(mods[0])
    elif len(mods) < 1:
        print('THIS DOES NOT HAVE MODELS')

#add the missing ones - surpassing loop:
#cents_list_hist.append('IPSL')
#mods_list_hist.append('IPSL-CM6A-LR')

mods_remove = []
cents_remove = []

for x in range(len(mods_list_s2)):
    if not mods_list_s2[x] in mods_list_hist:
        mods_remove.append(mods_list_s2[x])
        cents_remove.append(cents_list_s2[x])
for x in range(len(cents_remove)):
    cents_list_s2.remove(cents_remove[x])
    mods_list_s2.remove(mods_remove[x])
    
mods_remove = []
cents_remove = []
for x in range(len(mods_list_hist)):
    if not mods_list_hist[x] in mods_list_s2:
        mods_remove.append(mods_list_hist[x])
        cents_remove.append(cents_list_hist[x])
for x in range(len(cents_remove)):
    cents_list_hist.remove(cents_remove[x])
    mods_list_hist.remove(mods_remove[x])

mods_list = []
cents_list = []
for x in range(len(mods_list_s2)):
    if mods_list_s2[x] in mods_list_hist:
        mods_list.append(mods_list_s2[x])
        cents_list.append(cents_list_s2[x])

#list ensemble members


'''
ens_list_s2 = []
if len(cents_list) == len(mods_list):
    for i in range(len(cents_list)):
        ens = os.listdir(f'/badc/cmip6/data/CMIP6/ScenarioMIP/{cents_list[i]}/{mods_list[i]}/ssp245/')
        if len(ens) >= 1:
            for e in range(len(ens)):
                if not '.' in ens[e]:
                    ens_list_s2.append(ens[e])
                    break
        elif len(ens) < 1:
            print('THIS DOES NOT HAVE ENSEMBLE MEMBERS')
else: print('CENTRES AND MODELS NOT SAME LENGTH')
'''
# changing  if not '.' in ens[e]:   to    if 'r1i' in ens[e]:

ens_list_s2 = []
if len(cents_list) == len(mods_list):
    for i in range(len(cents_list)):
        ens = os.listdir(f'/badc/cmip6/data/CMIP6/ScenarioMIP/{cents_list[i]}/{mods_list[i]}/ssp245/')
        if len(ens) >= 1:
            for e in range(len(ens)):
                if 'r1i' in ens[e]:
                    ens_list_s2.append(ens[e])
                    break
        elif len(ens) < 1:
            print('THIS DOES NOT HAVE ENSEMBLE MEMBERS')
else: print('CENTRES AND MODELS NOT SAME LENGTH')

    
#list ensemble members

ens_list_hist = []
if len(cents_list) == len(mods_list):
    for i in range(len(cents_list)):
        ens = os.listdir(f'/badc/cmip6/data/CMIP6/CMIP/{cents_list[i]}/{mods_list[i]}/historical/')
        if ens_list_s2[i] in ens:
            ens_list_hist.append(ens_list_s2[i])
        else:
            print(i)
ens_list = ens_list_hist

mods_list_tas, cents_list_tas, ens_list_tas = mods_list, cents_list, ens_list


ds_list_s2 = []
ds_list_hist = []

dss = [ds_list_s2, ds_list_hist]
scen = ['ssp245', 'historical']

#adding UKESM and CESM at the end 
for x in range(2):
    
    for i in range(len(cents_list)):
       # print(i)
        path = f'/badc/cmip6/data/CMIP6/*MIP/{cents_list[i]}/{mods_list[i]}/{scen[x]}/{ens_list[i]}/Amon/tas/*/latest/'
        ds = rename_cmip6(xr.open_mfdataset(path+'*.nc'))
        dss[x].append(ds)

    path = f'/badc/cmip6/data/CMIP6/*MIP/MOHC/UKESM1-0-LL/{scen[x]}/r10i1p1f2/Amon/tas/gn/latest/'
    ds = rename_cmip6(xr.open_mfdataset(path+'*.nc'))
    dss[x].append(ds)
    
    path = f'/badc/cmip6/data/CMIP6/*MIP/NCAR/CESM2-WACCM/{scen[x]}/r1i1p1f1/Amon/tas/gn/latest/'
    ds = rename_cmip6(xr.open_mfdataset(path+'*.nc'))
    dss[x].append(ds)

def get_ITD_multi(da_tas): 
    weights = np.cos(np.deg2rad(da_tas.y))
    NH = da_tas.sel(y=slice(0, 90)).weighted(weights).mean(dim=['x', 'y'])
    SH = da_tas.sel(y=slice(-90, 0)).weighted(weights).mean(dim=['x', 'y'])
    ITD = NH-SH
    NH = NH.groupby('time.year').mean('time')
    SH = SH.groupby('time.year').mean('time')
    ITD = ITD.groupby('time.year').mean('time')
    GMT = (NH+SH)/2
    pattern = da_tas.groupby('time.year').mean('time')
    return ITD, NH,SH,GMT #, pattern


ITDs = []
NHs = []
SHs = []
GMTs = []
#patterns = []

for ds in range(len(ds_list_hist)):
    ds_con = xr.concat([ds_list_hist[ds], ds_list_s2[ds]], dim = 'time')
    itd, NH, SH, GMT = get_ITD_multi(ds_con.tas)
    ITDs.append(itd)
    NHs.append(NH)
    SHs.append(SH)
    GMTs.append(GMT)
 #   patterns.append(pattern)

ITDs_mean = []
for i in ITDs:
    means = i.rolling(year=10, center=True).mean().dropna("year")
    ITDs_mean.append(means)

NHs_mean = []
for i in NHs:
    means = i.rolling(year=10, center=True).mean().dropna("year")
    NHs_mean.append(means)

SHs_mean = []
for i in SHs:
    means = i.rolling(year=10, center=True).mean().dropna("year")
    SHs_mean.append(means)

GMTs_mean = []
for i in GMTs:
    means = i.rolling(year=10, center=True).mean().dropna("year")
    GMTs_mean.append(means)


patterns = []

for ds in range(len(ds_list_hist)):
    ds_con = xr.concat([ds_list_hist[ds], ds_list_s2[ds]], dim = 'time')
    ds_con_yrly = ds_con.groupby('time.year').mean('time')
    pattern = ds_con_yrly.rolling(year=10, center=True).mean().dropna("year")
    patterns.append(pattern)
    

patterns_mean = []
for i in patterns:
    means = i.rolling(year=10, center=True).mean().dropna("year")
    patterns_mean.append(means)

data_list = [ITDs_mean, NHs_mean, SHs_mean, GMTs_mean]
names = ['ITDs', 'NHs', 'SHs', 'GMTs']

# Define output directory
output_dir = "inter_files"
os.makedirs(output_dir, exist_ok=True)  # Create the folder if it doesn't exist

# Save each DataArray in the list
for data in range(len(data_list)):
    dirpath = os.path.join(output_dir, names[data])
    os.makedirs(dirpath, exist_ok=True)
    for i in range(len(data_list[data])):
        filename = f"{names[data]}_{i+1}.nc"
        filepath = os.path.join(dirpath, filename)
    
        # Save the DataArray to NetCDF format
        data_list[data][i].to_netcdf(filepath)


data_list = [patterns]
names = ['patterns']

output_dir = "inter_files"
os.makedirs(output_dir, exist_ok=True)  # Create the folder if it doesn't exist

# Save each DataArray in the list
for data in range(len(data_list)):
    dirpath = os.path.join(output_dir, names[data])
    os.makedirs(dirpath, exist_ok=True)
    for i in range(len(data_list[data])):
        filename = f"{names[data]}_{i+1}.nc"
        filepath = os.path.join(dirpath, filename)
    
        # Save the DataArray to NetCDF format
        data_list[data][i].to_netcdf(filepath)


/opt/jaspy/lib/python3.11/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/opt/jaspy/lib/python3.11/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
/opt/jaspy/lib/python3.11/site-packages/xarray/conventions.py:286: SerializationWarning: variable 'tas' has multiple fill values {1e+20, 1e+20} defined, decoding all values to NaN.
  var = coder.decode(var, name=name)
